In [13]:
import pickle
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer

class DenseTransformer(MinMaxScaler):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [14]:
df = pd.read_csv('text_store_label.csv')
df = df.fillna(" ")
u_class = df['store'].values
l  = df['link'].values
X  = df['text'].values
y  = df['label'].values

In [15]:
text_clf = Pipeline([
    ('tfidf', CountVectorizer()),
    ('tranf', DenseTransformer()),
    ('clf', RandomForestClassifier(n_estimators=200, n_jobs=3)),
])
text_clf.fit(l, y)
filename = 'link_clf2.sav'
pickle.dump(text_clf, open(filename, 'wb'))

In [18]:
text_clf.predict(l)

array([0, 0, 0, ..., 1, 1, 1])